In [16]:
:opt no-lint

# Partial Evaluation

부분계산(partial evaluation)이라는 개념을 이용해 인터프리터가 있다면 이를 이용해 컴파일러를 생성해 낼 수 있다.

이를 **후타무라 투영**(Futamura projection)이라고 한다. https://fi.ftmr.info/

실제로 현업에서 활용되는 컴파일러로는 PyPy라는 파이썬으로부터 실행파일을 만들어내는 컴파일러를 작성하는 데 바로 이 아이디어가 활용된 것으로 알려져 있다.


## Partial Evaluator
일단 부분계산을 실행하는 부분계산기(partial evualuator, $\textit{PE}\,$)의 개념부터 알아보자

$prog(I_S\uplus I_D) = v$ 일 때

$\textit{PE}(prog, I_S) = prog^*$ 단, $prog^*(I_D) = v$

예를 들어 $prog$가 $3 + x + y$에 해당하고 $I_S = \{x\mapsto 2\}$라면

$prog^*$는 $5 + y$에 해당한다.

-------
지금까지는 언어 종류는 대략 두가지 정도 부류로 컴파일러를 어떻게 만들까 살펴봤는데, 많이 단순화해서 입력은 고려하지 않았음.

입력(사용자 키보드 입력, 데이터베이스 읽어오기, 파일 읽어오기, 네트웍 등등)이 전혀 없으면 사실 인터프리터를 매번 돌릴 필요가 없다.

1 ~ 100 까지 더하는 프로그램을 반복문을 돌려서 작성했다면 (그런데 입력이 없고 정해져 있다면)
사실은 한번만 돌려보고 결과인 5050을 원래 프로그램 대신에 앞으로는 사용하면 되죠.
아무리 복잡한 프로그램도 입력이 전혀 없으면 (확률적으로 랜덤하게 실행때마다 다른 값이 나와야 하는 게 아니라면)
한번만 돌려보고 다음부터는 그 결과값을 바로 그냥 사용하면 굳이 계산을 매번 돌릴 필요가 없다는 거.

그래서 사실은 입력이 없는 프로그램은 그냥 상수로 최적화가 되어버립니다.


비유자하자면 입력이 없는 경우라면
 * 1 ~ 100 까지 더하는 프로그램을 반복문을 돌리는 <==== 인터프리터
 * 5050 <=== 컴파일한 실행파일이 내놓으면 되는 결과

보통은 1 ~ $n$ 까지 더하는 프로그램을 작성해서 $n$은 "입력"으로 생각하죠.
그 입력에 따라 다른 결과값을 프로그램을 실행해서 알아내는 게 보통 프로그램을 활용하는 방법.

예를 들어 $m$ ~ $n$ 까지 더하는 프로그램이라면 해놓고 $m$은 프로그램 실행 전에 항상 똑같이 미리 정해 놓는 상수라면,
$m$에 값을 지정하는 것이 정적 입력($I_S$)에 해당하고 나머지 $n$은 프로그램 실행 중에 입력받는 (혹은 실행할 때마다 다르게 지정할 수 있는) 동적 입력($I_D$)으로 처리한다고 보면 된다.

In [17]:
type Nm = String

data Exp = Var Nm      -- x
         | Lit Int     -- n
         | Add Exp Exp -- e1 + e2
         deriving Show

type Val = Int
type Env = [(Nm,Val)]

In [18]:
-- interpreter
eval :: Exp -> Env -> Val
eval (Var x)     env = case lookup x env of
                        Nothing -> error $ x ++ " not found"
                        Just v  -> v
eval (Lit n)     env = n
eval (Add e1 e2) env = eval e1 env + eval e2 env

In [26]:
e1 = Add (Add (Lit 3) (Var "x")) (Var "y")
iS = [("x",2)]
iD = [("y",10)]

eval e1 (iS++iD)

15

In [28]:
-- partial evaluator
pe :: Exp -> Env -> Exp
pe (Var x) env     = case lookup x env of
                        Nothing -> Var x
                        Just v  -> Lit v
pe (Lit n) env     = Lit n
pe (Add e1 e2) env = case (pe e1 env, pe e2 env) of
                        (Lit n1, Lit n2) -> Lit (n1 + n2)
                        (e1'   , e2'   ) -> Add e1' e2'

In [29]:
pe e1 iS

Add (Lit 5) (Var "y")

----
이건 매무 매우 간단한 경우고, 함수를 처리할 수 있는 람다식이 포함되면 `pe`가 조금 더 생각해야 될 게 있다.
왜냐하면 값(`Val`)이 단순히 정수가 아니고 클로저를 포함할 수 있게 되기 때문.

후타무라 투영 이야기까지 가려면 우리가 다루는 언어(`Exp`)가 좀더 많은 기능을 가정해야 설명하기 좋다.

예를 들면 `Exp` 언어의 인터프리터와 부분계산기를 `Exp` 언어로 작성 가능할 정도로 조금 더 본격적인 언어가 필요.

예를 들면 문자열 같은 걸 입력받아서 "3 + x + y"라는 걸 입력받으면 이걸 처리해서 계산할 수 있는 인터프리터와 부분계산기를
하스켈이 아니라 `Exp` 언어 자체로 작성할 수 있어야 한다.

------

`Exp` 언어에 충분히 많은 기능을 추가해서
`Exp` 자신의 언어에 대한 **인터프리터**와 **부분계산기**를 작성할 수 있다고 합시다.
그러면 인터프리터와 부분계산기를 이용해 컴파일러를 유도해 낼 수 있다.
이런 것을 발견한 사람이 후타무라 교수.

* $P_i$ 인터프리터 프로그램
* $PE$ 부분계산기 프로그램

$P_i$는 소스코드와 그 소스코드를 실행하면서 받는 *소스코드에 대한 입력*를 입력으로 받아서 결과값을 계산하는 프로그램.
그러니까 $P_i$ 입장에서는
소스코드가 *정적 입력*에 해당하고
그 소스코드를 실행하면서 소스코드에 넘기는 입력 부분을 *동적 입력*이라고 생각할 수 있다.

$P_i(source \uplus input) = result$

인터프리터를 부분계산하는데 소스코드를 정적입력으로 넘긴다고 생각하면
$PE(P_i, source) = exe$ 이렇게 나온 프로그램은 $exe(input) = result$ 가 나오는 실행파일에 해당한다.

여기서 잘 생각해 보면 소스코드를 동적 입력으로 받아서 실행파일에 해당하는 것을 만드는 건 *컴파일러*잖아?

그러니까 부분계산기 자체를 *인터프리터를 정적 입력으로 취급해서* 부분계산해 버리면 컴파일러가 나오겠네???!!!!

$PE(PE, P_i) = P_c$ 그러면 이렇게 나온 프로그램은 $P_c(source) = exe$ 이런 식으로 동작한다.

그러니까 $source$로부터 그 소스코드를 인터프리터로 실행했을 때와 같은 결과를 얻는 실행파일에 해당하는 $exe$를 만들어내는
프로그램이 바로 $P_c$라는 것. 그런데 그렇게 동작하는 $P_c$라는 프로그램이 바로 컴파일러라는 것에 기대하는 것과 정확히 똑같은 일을 한다. 

---
여기서 아 그렇구나 하고 넘어갈 수도 있지만 ...

한가지 더 생각해야 되는 부분이 최초에는 인터프리터($P_i$)나 부분계산기를($PE$)를 어떻게 실행할까?

자기 자신의 언어로 $PE$와 $P_i$를 작성했다는 거는 이게 정말 처음에는 이 언어의 구현이 존재하지 X.

그러니까 최초에는 다른 언어로 작성할 수밖에 없다. <=== 부트스트랩

컴파일러 유도를 위해서는 $PE$만 실행할 수 있으면 되지만 현실적으로 $P_i$도 제대로 작성했나 테스트를 해보려면 현실적으로는 실행을 안 해 볼 수는 없겠죠. 일단 실행가능한 인터프리터($P_i$)가 있으면 $PE$도 그 인터프리터를 이용해서 실행가능하겠죠.

앞서 실제로 PyPy라는 파이썬 컴파일러가 이런 방식으로 만들어졌다고 했는데
이것도 부트스트랩은 CPython이라는 C로 작성된 파이썬 인터프리터를 활용해서 부트스랩을 한 것.
그 다음에 파이썬으로 파이썬의 인터프리터나 부분계산기를 작성해서 컴파일러 유도.

----

----

# 좀더 자세한 관련 내용은 다음 자료를 참고

http://www.cs.ioc.ee/ewscs/2016/geuvers/geuvers-slides-lecture2.pdf

 - 강의노트 형식
 - 순수한 람다계산법만으로 람다계산법 자신의 인터프리터를 작성하는 방법을 설명

https://www.cs.tufts.edu/comp/150FP/archive/neil-jones/mix-partial-evaluator.pdf
 - 논문
 - 앞서 대략 설명한 아이디어를 좀더 구체적으로 정리해 그대로 따라 구현할 수 있을 정도로 자세히 정리
 - 네 가지 다른 언어를 사용할 수 있을 가능성
    * L 언어로 인터프리터를 작성
    * P 언어로 부분계산기를 작성 (부분계산기의 입력으로 L 언어의 프로그램 소스코드가 들어감)
    * S 언어는 처리하고자 하는 대상 언어 (인터프리터, 컴파일러의 입력으로 S 언어 소스코드가 들어감)
    * T 언어는 컴파일러의 결과로 나오는 대상(혹은 목적) 코드의 언어. 논문에서는 T = L 로 놓고 진행함.
    
L = S 일 때 self-interpretation 또는 metacircular interpreter라고 부르기도 한다.

예를 들어 파이썬의 컴파일러를 이런 식으로 유도한다면 S언어는 당연히 파이썬일 거고,
L언어는 C를 활용할 수도 있겠죠. C로 만든 CPython이라는 가장 대중적인 인터프리터가 이미 존재하니까.